# Import Libraries

In [4]:
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.datasets import boston_housing
from keras import layers
from keras.initializers import Initializer
from sklearn.cluster import KMeans
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# Definition of the RBF Layer
The code is based on the implementation of Petra Vidnerova et. all, which can be found [here](https://github.com/PetraVidnerova/rbf_keras). Also, the [Keras Tutorial on creating a custom layer](https://www.tutorialspoint.com/keras/keras_customized_layer.htm) is followed.

In [5]:
class RBFLayer(layers.Layer):
    # output_dim: number of hidden units (number of outputs of the layer)
    # initializer: instance of initializer to initialize centers
    # betas: float, initial value for betas (beta = 1 / 2*pow(sigma,2))
    def __init__(self, output_dim, initializer, betas=1.0, **kwargs):
        self.betas = betas
        self.output_dim = output_dim
        self.initializer = initializer
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers', shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer, trainable=False)
        sigma = np.zeros(self.output_dim)
        for i in range(0, self.output_dim):
            d_max = 0
            for j in range(0, self.output_dim):
                d = np.linalg.norm(self.centers[i] - self.centers[j])
                if d > d_max:
                    d_max = d
            sigma[i] = d_max / np.sqrt(2 * self.output_dim)
        self.betas = np.ones(self.output_dim) / (2 * (sigma ** 2))
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs, *args, **kwargs):
        C = tf.expand_dims(self.centers, -1)  
        H = tf.transpose(C - tf.transpose(inputs))  
        return tf.exp(-self.betas * tf.math.reduce_sum(H ** 2, axis=1))

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.output_dim

## KMeans Definition for the Centers
Petra Vinderova also implements the [KMeans Initialiser](https://github.com/PetraVidnerova/rbf_keras/blob/master/kmeans_initializer.py)

In [6]:
class InitCentersKMeans(Initializer):
    def __init__(self, X, max_iter=100):
        self.X = X
        self.max_iter = max_iter
        super().__init__()

    def __call__(self, shape, dtype=None, *args):
        assert shape[1] == self.X.shape[1]
        n_centers = shape[0]
        km = KMeans(n_clusters=n_centers, max_iter=self.max_iter, verbose=0)
        km.fit(self.X)
        return km.cluster_centers_


## Definition of the necessary metrics
The following metrics are based on these implementations:
- [R2 score](https://jmlb.github.io/ml/2017/03/20/CoeffDetermination_CustomMetric4Keras/)
- [Root Mean Squared Error](https://stackoverflow.com/questions/43855162/rmse-rmsle-loss-function-in-keras) / and this [issue](https://github.com/keras-team/keras/issues/10706)
- [Mean Squared Error](https://stackoverflow.com/questions/67216102/keras-mean-squared-error-mse-calculation-definition-for-images)

In [7]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))


def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def r2_score(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - SS_res/(SS_tot + K.epsilon())

## Other Functions that will be used

In [8]:
def data_normalisation(data):
    encoder = StandardScaler()
    return encoder.fit_transform(data)

def split_validation(data, length_training, length_validation):
    data_train, data_validation = tf.split(data, [length_training, length_validation], 0)
    return data_train, data_validation

## Data Preprocessing

In [ ]:
test_length = 0.25
(x_train, y_train), (x_test, y_test) = boston_housing.load_data(test_split=test_length)

normalise = True
if normalise:
    x_train = data_normalisation(x_train)
    x_test = data_normalisation(x_test)
    
# length_validation = round(0.2 * len(x_train))
# length_training = round(0.8 * len(x_train))
# x_train, x_validation = split_validation(x_train, length_training, length_validation)
# y_train, y_validation = split_validation(y_train, length_training, length_validation)

# x = tf.concat([x_train, x_validation], 0).numpy()
# y = tf.concat([y_train, y_validation], 0).numpy()
# print(len(x))

## Hyperparameters

In [10]:
times = [] 
k = 5  # 5-fold cross validation
learning_rate = 0.001
epochs = 100
batch_size = 256

neurons_percentage = [0.05, 0.15, 0.3, 0.5]
total_neurons_rbf = [round(n*0.8*x_train.shape[0]) for n in neurons_percentage]
total_neurons_hidden2 = [32, 64, 128, 256]
total_dropout_probabilities = [0.2, 0.35, 0.5]
neurons_output = 1

grid = []
index = 0
start = time.time()
rmse_final = []

## Define the baseline model

In [11]:
def baseline_model(x_train, y_train, x_val, y_val, neurons_rbf=128, neurons_h2=128, neurons_output=1, drop_prob=0, batch_size=256, epochs=100):
    model = Sequential()
    model.add(RBFLayer(neurons_rbf, initializer=InitCentersKMeans(x_train), input_shape=(13, )))
    model.add(Dense(neurons_h2))
    model.add(Dropout(drop_prob))
    model.add(Dense(neurons_output))

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate), loss=mean_squared_error,metrics=[r2_score, root_mean_squared_error])
    fit_model = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val),verbose=0)
    return fit_model, model



## Fine Tuning

In [ ]:
for dropout_probability in total_dropout_probabilities:
    for neurons_hidden2 in total_neurons_hidden2:
        n = 0
        for neurons_rbf in total_neurons_rbf:
            rmse = []
            kf = KFold(k, shuffle=True, random_state=42)
            for train, test in kf.split(x_train):
                x_fold_train = x_train[train]
                y_fold_train = y_train[train]
                x_fold_validation = x_train[test]
                y_fold_validation = y_train[test]
                
                fit_model, model = baseline_model(x_fold_train, y_fold_train, x_fold_validation, y_fold_validation, neurons_rbf, neurons_hidden2, neurons_output,
                                          dropout_probability, batch_size, epochs)
                rmse.append(min(fit_model.history['root_mean_squared_error']))

            rmse_final.append(np.mean(rmse))
            grid.append((neurons_percentage[n], dropout_probability, neurons_hidden2, neurons_rbf))
            n +=1

## Find the best result

In [ ]:
# times.append((time.time()-start))
# print(f'Time needed: '+str(times[0])
x = 3
print(x)
# min_value_of_loss_function = min(rmse_final)
# index_of_min = [i for i, value in enumerate(rmse_final) if value == min_value_of_loss_function]
# min_rmse = rmse_final[index_of_min]
# best_rbf_neuron_per = grid[index_of_min][0]
# best_drop_prob = grid[index_of_min][1]      
# best_h2_neuron = grid[index_of_min][2]  
# best_rbf_neuron = grid[index_of_min][3]
# print(f'Min RMSE: '+str(rmse_final[i]))
# print(f'Percentage of neurons in RBF Layer: '+str(100*best_rbf_neuron_per)+'%')
# print(f'Number of neurons in RBF Layer: '+str(best_rbf_neuron))
# print(f'Number of neurons in second hidden Layer: '+str(best_h2_neuron))
# print(f'Dropout Propability: '+str(best_drop_prob))

## Run the optimised model

In [ ]:
length_validation = round(0.2 * len(x_train))
length_training = round(0.8 * len(x_train))
x_train, x_validation = split_validation(x_train, length_training, length_validation)
y_train, y_validation = split_validation(y_train, length_training, length_validation)


fit_model, model = baseline_model(x_train, y_train, x_validation, y_validation, best_rbf_neuron, best_h2_neuron, neurons_output, 
                          best_drop_prob)
score = model.evaluate(x_test, y_test)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), loss=mean_squared_error,
              metrics=[r2_score, root_mean_squared_error])
history = model.fit(Trnx, Trny, batch_size=4, epochs=100, validation_data=(valx, valy))
score = model.evaluate(Tstx, Tsty)

train_loss = [fit_model.history['loss'][i] for i in range(100)]
validation_loss = [fit_model.history['val_loss'][i] for i in range(100)]

train_r2 = fit_model.history['r2_score']
validation_r2 = fit_model.history['val_r2_score']

train_rmse = fit_model.history['root_mean_squared_error']
validation_rmse = fit_model.history['val_root_mean_squared_error']

plt.plot(train_loss)
plt.plot(validation_loss)

plt.title('Learning Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.grid()
plt.show()

plt.plot(train_r2)
plt.plot(validation_r2)

plt.title('R2')
plt.xlabel('Epochs')
plt.ylabel('R2')
plt.legend(['Training R2', 'Validation R2'], loc='lower right')
plt.grid()
plt.show()

plt.plot(train_rmse)
plt.plot(validation_rmse)

plt.title('RMSE')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.legend(['Training RMSE', 'Validation RMSE'], loc='lower right')
plt.grid()
plt.show()